In [ ]:
import demjson
import openai
import pandas as pd
import datetime
import os

In [ ]:
class GraspPartDetector:
    def __init__(self, api_key=os.environ["OPENAI_API_KEY"]):
        openai.api_key = api_key
        self.dialogue_df = pd.DataFrame(columns=["input", "output"])
        self.now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

    def receive_response_from_API(self, object_nm, background=""):
        content = f"""
        Please provide a breakdown of the {object_nm} into several parts that the robot can grasp to transfer the object. Which part should be grasped to ensure a stable grip?
        The object is placed horizontally on a foundation, and the robot grasps the object from above.
        The robot is fixed on the foundation.
        The robot has a parallel gripper.
        Ensure a non-slip grip when performing the grasping.
        """
        message_list = [
            {
                "role": "system",
                "content": "You are a multi-axis robot with a parallel gripper, skilled at stable object grasping.",
            },
        ] + [
            {"role": "user", "content": content},
        ]
        if background != "":
            message_list += [
                {"role": "system", "content": background},
            ]
        response = (
            openai.ChatCompletion.create(
                model="gpt-4-0613",
                messages=message_list,
                temperature=0,
            )
            .choices[0]["message"]["content"]
            .strip()
        )
        print(response)

        convert_into_json = """
        Respond in the following JSON format, indicating the most suitable part and providing a brief explanation for your selection.:
        {
        "most suitable part": "part name",
        "explanation": "Please provide a brief explanation for your selection."
        }
        """

        message_list += [
            {"role": "assistant", "content": response},
        ] + [{"role": "user", "content": convert_into_json}]

        response = (
            openai.ChatCompletion.create(
                model="gpt-4-0613",
                messages=message_list,
                temperature=0,
            )
            .choices[0]["message"]["content"]
            .strip()
        )
        output_json = self.load_json_str(response)

        self.dialogue_df.loc[len(self.dialogue_df)] = [object_nm, output_json]
        self.dialogue_df.to_csv(f"../data/dialogue_{self.now}.csv")
        return response

    def load_json_str(self, res_json_str):
        try:
            json_data = demjson.decode(res_json_str)
            return json_data
        except demjson.JSONDecodeError as e:
            print("Invalid JSON format:", e)

In [ ]:
gpd = GraspPartDetector()
object_nm = "kitchen shears"
output_json = gpd.receive_response_from_API(object_nm)

In [ ]:
print(output_json)